In [23]:
import pandas as pd
import numpy as np
import json
from datetime import datetime
from time import time

In [24]:
covid_data = pd.read_csv("data/district.csv")

In [25]:
covid_data

,Date,State,District,Confirmed,Recovered,Deceased,Other,Tested
0,2020-04-26,Andaman and Nicobar Islands,Unknown,33,11,0,0,NaN
1,2020-04-26,Andhra Pradesh,Anantapur,53,14,4,0,NaN
2,2020-04-26,Andhra Pradesh,Chittoor,73,13,0,0,NaN
3,2020-04-26,Andhra Pradesh,East Godavari,39,12,0,0,NaN
4,2020-04-26,Andhra Pradesh,Guntur,214,29,8,0,NaN
...,...,...,...,...,...,...,...,...
319085,2021-09-01,West Bengal,Purba Bardhaman,40613,40161,187,0,NaN
319086,2021-09-01,West Bengal,Purba Medinipur,61697,60774,382,0,NaN
319087,2021-09-01,West Bengal,Purulia,19280,19118,113,0,NaN
319088,2021-09-01,West Bengal,South 24 Parganas,97737,95765,1287,0,NaN


In [26]:
#Removing addition column
col = ["State","Recovered","Deceased","Other","Tested"]
covid_data.drop(col,axis=1,inplace=True)

In [27]:
#reading neighbors json file as disctionary
with open('data/neighbors-districts-modified.json') as json_file:
    neighbor_data = json.load(json_file)

In [28]:
#Removing noisy data
neighbor_districts_data = []
neighbor_districts = list(neighbor_data.keys())

neighbor_districts.remove("HP_Hamirpur")
neighbor_districts.remove("CT_Bilaspur")
neighbor_districts.remove("RJ_Pratapgarh")
neighbor_districts.remove("CT_Balrampur")
neighbor_districts.remove("BR_Aurangabad")

l = []

for i in range(len(neighbor_districts)):
    dist = neighbor_districts[i].split("_")[1]
    l.append(dist)
    neighbor_districts_data.append((dist,neighbor_districts[i]))

    
covid_districts = np.array(covid_data["District"])
covid_districts_unique = np.unique(covid_districts)
len(covid_districts_unique)

ncom = []
for x in covid_districts_unique:
    if x not in l:
        ncom.append(x)

#removing ncom district data from covid data

for x in ncom:
    index_names = covid_data[ covid_data['District'] == x ].index
    covid_data.drop(index_names,inplace=True)

#reseting index value
covid_data.reset_index(inplace = True,drop=True)

In [29]:
date_list = pd.date_range(start = "26-04-2020",end = "14-08-2021").to_pydatetime().tolist()
for i in range(len(date_list)):
    date_list[i] = date_list[i].strftime("%Y-%m-%d")
    
#week_list contain last date of a week
week_list = []
for i in range(6,len(date_list)+1,7):
    week_list.append(date_list[i])

# Program to generate case-week.csv   

In [30]:
#dfs is array of dataframes of each date in week list
dfs = []
for x in week_list:
    temp = covid_data.query("Date==@x")
    dfs.append(temp)

#arranging the dataframes in sorted order according to district name for faster execution 
for i in range(len(week_list)):
    dist = list(dfs[i]["District"])
    for x in neighbor_districts_data:
        
        if x[0] in dist:
            pass
        else:
            dic = {
                "Date":list(dfs[i]["Date"])[0],
                "District":x[0],
                "Confirmed":0
            }
            dfs[i] = dfs[i].append(dic,ignore_index=True)
    dfs[i] = dfs[i].sort_values("District")
    
#sorted district list
neighbor_districts_data.sort(key=lambda x: x[0])

In [31]:
#cumulative cases per week per district
no_of_districts = len(neighbor_districts_data)
cumulative_cases = []
for i in range(6):
    cumulative_cases.append([0]*no_of_districts)

for i in range(len(week_list)):
    cumulative_cases.append(list(dfs[i]["Confirmed"]))

In [32]:
present_cases = []
for i in range(no_of_districts):
    present_cases.append(0)
for i in range(1,len(cumulative_cases)):
    for j in range(no_of_districts):
        r = cumulative_cases[i][j] - cumulative_cases[i-1][j]
        if r<0 : r=0
        present_cases.append(r)

In [33]:
#list of time nd corresponding district
timeid = []
for i in range(len(week_list)+6):
    for j in range(no_of_districts):
        timeid.append(i+1)
        
districtid = []
for i in range(len(week_list)+6):
    for x in neighbor_districts_data:
        districtid.append(x[1])

In [34]:
#Resultant datafram
result = {
    "districtid" :districtid,
    "timeid" : timeid,
    "cases" : present_cases
}
result = pd.DataFrame(result)

In [40]:
result = result.sort_values(["districtid","timeid"])

In [41]:
result

,districtid,timeid,cases
22,AP_Anantapur,1,0
722,AP_Anantapur,2,0
1422,AP_Anantapur,3,0
2122,AP_Anantapur,4,0
2822,AP_Anantapur,5,0
...,...,...,...
48962,WB_Uttar Dinajpur,70,65
49662,WB_Uttar Dinajpur,71,66
50362,WB_Uttar Dinajpur,72,45
51062,WB_Uttar Dinajpur,73,66


In [99]:
#exporting the created file
result.to_csv("case-week.csv",index=False)

# Program to generate case-month.csv 

In [42]:
#creating list of months
month_list = []
for d in date_list:
    if "14" in d:
        month_list.append(d)

In [43]:
#dfs is array of dataframes of each date in month list
dfs = []
for x in month_list:
    temp = covid_data.query("Date==@x")
    dfs.append(temp)
#arranging the dataframes in sorted order according to district name for faster execution 
for i in range(len(month_list)):
    dist = list(dfs[i]["District"])
    for x in neighbor_districts_data:
        
        if x[0] in dist:
            pass
        else:
            dic = {
                "Date":list(dfs[i]["Date"])[0],
                "District":x[0],
                "Confirmed":0
            }
            dfs[i] = dfs[i].append(dic,ignore_index=True)
    dfs[i] = dfs[i].sort_values("District")
    

In [44]:
#cumulative cases per week per district
cumulative_cases = []
for i in range(1):
    cumulative_cases.append([0]*700)

for i in range((len(month_list))):
    cumulative_cases.append(list(dfs[i]["Confirmed"]))

In [45]:
present_cases = []
for i in range(700):
    present_cases.append(0)
for i in range(1,len(cumulative_cases)):
    for j in range(700):
        r = cumulative_cases[i][j] - cumulative_cases[i-1][j]
        if r<0 : r=0
        present_cases.append(r)

In [46]:
#list of time nd corresponding district
timeid = []
for i in range(len(month_list)+1):
    for j in range(700):
        timeid.append(i+1)
        
districtid = []
for i in range(len(month_list)+1):
    for x in neighbor_districts_data:
        districtid.append(x[1])

In [47]:
#Resultant datafram
result = {
    "districtid" :districtid,
    "timeid" : timeid,
    "cases" : present_cases
}
result = pd.DataFrame(result)

In [48]:
result = result.sort_values(["districtid","timeid"])

In [49]:
result

,districtid,timeid,cases
22,AP_Anantapur,1,0
722,AP_Anantapur,2,118
1422,AP_Anantapur,3,345
2122,AP_Anantapur,4,3188
2822,AP_Anantapur,5,24445
...,...,...,...
9062,WB_Uttar Dinajpur,13,598
9762,WB_Uttar Dinajpur,14,6304
10462,WB_Uttar Dinajpur,15,4896
11162,WB_Uttar Dinajpur,16,724


In [107]:
#exporting the file
result.to_csv("case-month.csv",index=False)

# Program to generate case-overall.csv

In [54]:
df = covid_data.query("Date=='2021-08-14'")

In [55]:
#overall cases 
dist = list(df["District"])
for x in neighbor_districts_data:
    if x[0] in dist:
        pass
    else:
        dic = {
            "Date":"2021-08-14",
            "District":x[0],
            "Confirmed":0
        }
        df = df.append(dic,ignore_index=True)
df = df.sort_values("District")

In [56]:
df.drop("Date",axis=1,inplace=True)
df.rename(columns = {"Confirmed":"Overall"})

,District,Overall
605,Adilabad,0
301,Agar Malwa,3303
498,Agra,25736
103,Ahmedabad,238044
255,Ahmednagar,297459
...,...,...
699,Yadadri Bhuvanagiri,0
239,Yadgir,27502
167,Yamunanagar,24571
288,Yavatmal,76223


In [111]:
df.to_csv("cases-overall.csv",index=False)